# Checking env and path context

In [1]:
import os, sys
sys.executable

'C:\\Users\\HP\\Anaconda3\\envs\\galicia\\python.exe'

In [2]:
os.getcwd()

'C:\\Users\\HP\\Charles\\Proyectos&Trabajos\\kaggle\\galicia'

In [3]:
!DIR

 Volume in drive C is Windows
 Volume Serial Number is 3C1E-D98C

 Directory of C:\Users\HP\Charles\Proyectos&Trabajos\kaggle\galicia

30/07/2019  20:50    <DIR>          .
30/07/2019  20:50    <DIR>          ..
22/07/2019  00:03    <DIR>          .ipynb_checkpoints
21/07/2019  20:46    <DIR>          banco-galicia-dataton-2019
21/07/2019  20:41        96.684.887 banco-galicia-dataton-2019.zip
30/07/2019  20:50             8.487 simple_lgbm.ipynb
               2 File(s)     96.693.374 bytes
               4 Dir(s)  262.383.230.976 bytes free


# Importing py packs

In [54]:
import pandas as pd
import numpy as np

from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Init variables

In [5]:
pageview_path = './banco-galicia-dataton-2019/pageviews/pageviews.csv'
conversions_path = './banco-galicia-dataton-2019/conversiones/conversiones.csv'

# Start analysis

## Pageview loading

In [6]:
pageview_data = pd.read_csv(pageview_path, parse_dates=['FEC_EVENT'])

In [7]:
pageview_data.head(5)

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID
0,2018-03-30 07:35:48,1,1,1,1,1,1,0
1,2018-03-30 07:35:52,2,2,2,2,2,1,0
2,2018-03-30 07:36:11,3,2,2,2,3,1,0
3,2018-03-30 07:36:16,4,2,2,2,3,1,0
4,2018-03-30 07:41:38,5,2,2,2,2,1,0


In [8]:
pageview_data['FEC_EVENT'].describe()

count                17936934
unique               10496454
top       2018-11-05 14:11:08
freq                       39
first     2018-01-01 00:09:17
last      2018-12-31 23:59:59
Name: FEC_EVENT, dtype: object

In [9]:
pageview_data.groupby(['USER_ID']).count().head(10)

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM
USER_ID,,,,,,,
0,6431,6431,6431,6431,6431,6431,6431
1,2111,2111,2111,2111,2111,2111,2111
2,2368,2368,2368,2368,2368,2368,2368
3,1935,1935,1935,1935,1935,1935,1935
4,3105,3105,3105,3105,3105,3105,3105
5,5878,5878,5878,5878,5878,5878,5878
6,1642,1642,1642,1642,1642,1642,1642
7,2562,2562,2562,2562,2562,2562,2562
8,2299,2299,2299,2299,2299,2299,2299


## Conversions loading

In [10]:
conversion_data = pd.read_csv(conversions_path)

In [11]:
conversion_data.head(5)

,mes,anio,USER_ID
0,7.0,2018.0,1410.0
1,8.0,2018.0,10755.0
2,8.0,2018.0,8270.0
3,10.0,2018.0,7558.0
4,9.0,2018.0,10731.0


In [12]:
conversion_data.mes = conversion_data.mes.astype(int) 
conversion_data.anio = conversion_data.anio.astype(int) 
conversion_data.USER_ID = conversion_data.USER_ID.astype(int) 

# Strategy taken

The approach taken for this time is to count number of visits from each user_id, cross it with conversions file and train a model according this only variable

In [13]:
print(pageview_data.columns)

Index(['FEC_EVENT', 'PAGE', 'CONTENT_CATEGORY', 'CONTENT_CATEGORY_TOP',
       'CONTENT_CATEGORY_BOTTOM', 'SITE_ID', 'ON_SITE_SEARCH_TERM', 'USER_ID'],
      dtype='object')


In [14]:
data = pageview_data.drop(['PAGE', 'CONTENT_CATEGORY', 'CONTENT_CATEGORY_TOP',
       'CONTENT_CATEGORY_BOTTOM', 'SITE_ID', 'ON_SITE_SEARCH_TERM'], axis='columns')

In [15]:
data.rename({'FEC_EVENT': 'NUMBER_VISITS'}, inplace=True)
data = data.groupby(['USER_ID']).count()
data.reset_index(inplace=True)

In [16]:
data.head(5)

,USER_ID,FEC_EVENT
0,0,6431
1,1,2111
2,2,2368
3,3,1935
4,4,3105


In [17]:
data = data.merge(conversion_data, how='left', on=['USER_ID'])

In [18]:
data.head(5)

,USER_ID,FEC_EVENT,mes,anio
0,0,6431,NaN,NaN
1,1,2111,NaN,NaN
2,2,2368,NaN,NaN
3,3,1935,NaN,NaN
4,4,3105,NaN,NaN


In [19]:
data['label'] = data.apply(lambda row: not pd.isnull(row['mes']), axis=1)

In [20]:
data['label'].value_counts()

False    10590
True      1738
Name: label, dtype: int64

In [21]:
data.set_index(['USER_ID'], inplace=True)

In [23]:
# train_x = data[data.mes < 10].copy()
# train_y = train_x['label'].copy()
# train_x.drop(['label', 'mes', 'anio'], axis='columns', inplace=True)

# test_x = data[data.mes >= 10].copy()
# test_y = test_x['label'].copy()
# test_x.drop(['label', 'mes', 'anio'], axis='columns', inplace=True)
y = data['label'].copy()
X = data.drop(['label','mes','anio'], axis='columns')

In [26]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = .3, random_state=42) 

In [27]:
print(train_y.value_counts())
print(test_y.value_counts())

False    7411
True     1218
Name: label, dtype: int64
False    3179
True      520
Name: label, dtype: int64


In [28]:
print(train_x.shape)
print(train_y.shape)

(8629, 1)
(8629,)


In [29]:
print(test_x.shape)
print(test_y.shape)

(3699, 1)
(3699,)


In [30]:
%%time
lgbm = LGBMClassifier(random_state=5)
lgbm.fit(train_x, train_y)

Wall time: 170 ms


In [31]:
pred_y = lgbm.predict(test_x)

In [32]:
accuracy_score(test_y, pred_y)

0.8594214652608814

In [44]:
y_probs = lgbm.predict_proba(test_x)
lgbm.classes_

array([False,  True])

In [48]:
scores = list()
for probs in y_probs: scores.append(probs[1])

In [52]:
results = test_x.copy()
results.drop(['FEC_EVENT'], axis='columns', inplace=True)
results['SCORE'] = scores

In [56]:
roc = roc_auc_score(test_y, scores)
print(roc)

0.6906075326058025
